In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
dev_df=pd.read_csv('./output/dev_hypothesis_evidences.csv')
dev_df

,hypothesis,evidences,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement: "", "" Event-free Surviv...",Contradiction
1,Patients with significantly elevated ejection ...,"["" Cardiac left ventricular function with res...",Contradiction
2,a significant number of the participants in th...,"["" Enterocolitis 1/167 (0.60%)"", "" Enterocol...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement: "", "" Local Control Usi...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"["" Prior treatment with a phosphatidylinosito...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1: "", "" Letrozole"", "" Partici...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", "" Total: 16/48 (33.33%)...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement: "", "" Number of Patient...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement: "", "" Progression-free ...",Entailment


In [3]:
hypothesis_lst=dev_df['hypothesis'].values.tolist()
len(hypothesis_lst)

200

In [4]:
evidence_lst=dev_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

200

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in dev_df[['hypothesis','evidences']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.512983,0.635036,0.500000,0.87,0.500
1,0.1,0.512983,0.557522,0.500000,0.63,0.500
2,0.2,0.512983,0.388235,0.471429,0.33,0.480
3,0.3,0.512983,0.320000,0.480000,0.24,0.490
4,0.4,0.512983,0.283582,0.558824,0.19,0.520
5,0.5,0.512983,0.209677,0.541667,0.13,0.510
6,0.6,0.512983,0.184874,0.578947,0.11,0.515
7,0.7,0.512983,0.123894,0.538462,0.07,0.505
8,0.8,0.512983,0.093458,0.714286,0.05,0.515
9,0.9,0.512983,0.057143,0.600000,0.03,0.505
